## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride.

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

In [44]:
!pip install pandas seaborn matplotlib scikit-learn pyarrow

In [45]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error

In [46]:
df = pd.read_parquet('Data/yellow_tripdata_2023-01.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19 <--

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

In [47]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [48]:
df.duration.std()

np.float64(42.59435124195458)

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59  <--
* 52.59
* 62.59

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [49]:
prev = len(df)
print("Before: ", prev)
df = df[(df.duration >= 1) & (df.duration <= 60)]
after = len(df)
print("After: ", after)
print("Proportion: ", 1-(prev-after)/prev)

Before:  3066766
After:  3009173
Proportion:  0.9812202822125979


What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%  <--

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

In [50]:
categorical = ['PULocationID', 'DOLocationID']

df[categorical] = df[categorical].astype(str)
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [51]:
X_train.shape

(3009173, 515)

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515  <--
* 715

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

In [52]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.649262183753913

What's the RMSE on train?

* 3.64
* 7.64  <--
* 11.64
* 16.64

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

In [54]:
df_val = pd.read_parquet('Data/yellow_tripdata_2023-02.parquet')
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
df_val[categorical] = df_val[categorical].astype(str)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_val[target].values

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.811812092681157

What's the RMSE on validation?

* 3.81
* 7.81   <--
* 11.81
* 16.81

In [5]:


df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)